# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [5]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [6]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [7]:
from new_key import IEX_CLOUD_API_TOKEN
print(IEX_CLOUD_API_TOKEN)

pk_679f66cec6ec4a47a25d4d9fae1d5045


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock

*We get the base url from the Testing Sandbox on the iexcloud.io/docs/api/ page, and from there we need to pick an endpoint that contains both the Market Cap and the Price of each stock. After a bit of searching, you realise that the Quote endpoint gives both the Market Cap and Price of the stock.*
*Need to use an f string for the placement of the symbol in the api_url.*
*We need to add our token to the end of the api_url to tell the api that we have permission to access the data it stores.*
*The data variable is of type Response.*

In [8]:
# Getting the symbol of the stock that we are intersting in, we are taking Apple as an example.
symbol = "aapl"
# Making the API URL including the name of the symbol and the IEX Token. 
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
# Getting the data from the database by making a http request. We turn the response object ot json object.
jsonResponse = requests.get(api_url).json()
# Since the responce is stored in a list, and we only want the dictionary
data = jsonResponse[0]


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [9]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [10]:
my_columns = ['Ticker','Stock Price','Market Capitalization','Number of Shares to Buy']
stock_dataframe = pd.DataFrame(columns=my_columns)
print(stock_dataframe)

Empty DataFrame
Columns: [Ticker, Stock Price, Market Capitalization, Number of Shares to Buy]
Index: []


We are going to append to the dataframe using a pandas series, rather than a using a list. A pandas dataframe is a 2D structure with rows and columns, while a series in a 1D structure with just rows. 
Pandas series accept python lists and can be used to append to a dataframe. 

In [11]:
newRow = pd.Series([symbol,price,market_cap,'N/A'], index=my_columns)
stock_dataframe = pd.concat([stock_dataframe, newRow], ignore_index=True)

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [12]:
finalDataframe = pd.DataFrame(columns=my_columns)
for i, stock in enumerate(stocks['Ticker']):
    newAPIurl = f'https://api.iex.cloud/v1/data/core/quote/{stock}?token={IEX_CLOUD_API_TOKEN}'
    try:
        newJsonResponse = requests.get(newAPIurl).json()
    except:
        continue
    data = newJsonResponse[0]
    finalDataframe.loc[i,'Ticker'] = stock
    finalDataframe.loc[i, 'Stock Price'] = data['latestPrice']
    finalDataframe.loc[i,'Market Capitalization'] = data['marketCap']
    finalDataframe.loc[i,'Number of Shares to Buy'] = 'N/A'
    

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

*In the video he nedds to split the list into lists of 100 stocks and then create a batch api call for each 100 stocks string. IEX has updated their batch api calls, and now allow upto 2000 calls to be called in a batch api call, so we can call for the entire list.*

In [13]:
def chunks(list, n):
    for i in range(0, len(list), n):
        yield list[i:i+n]

In [14]:
# Creating final dataframe to hold all ths stocks that are in the IEX database.
stock_dataframe = pd.DataFrame(columns=my_columns)
# Making a dataframe using the Ticker column that would hold all the stocks not in the database. 
dne_dataframe = pd.DataFrame(columns= ["Ticker"])
ticker_list = stocks['Ticker']
# Making the final_stock_list.
symbol_groups = list(chunks(ticker_list, 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
ticker_number = 0
for symbol_string in symbol_strings:
    # Making the batch api call url.
    batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&range=1m&token={IEX_CLOUD_API_TOKEN}'
    # Making the request using the batch api url above.
    json_data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        try:
            stock_dataframe.loc[ticker_number, 'Ticker'] = symbol
            stock_dataframe.loc[ticker_number, 'Stock Price'] = json_data[symbol]['quote']['latestPrice']
            stock_dataframe.loc[ticker_number, 'Market Capitalization'] = json_data[symbol]['quote']['marketCap']
            stock_dataframe.loc[ticker_number, 'Number of Shares to Buy'] = 'NaN'
            ticker_number += 1
        except KeyError:
            dne_dataframe.loc[ticker_number, 'Ticker'] = symbol
            ticker_number += 1
# Deleting the index that leads to NaN stock price and market cap from the dataframe.
rows_to_remove = dne_dataframe.index
stock_dataframe = stock_dataframe.drop(stock_dataframe.index[rows_to_remove])

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

*The way this is oging to work is the number of shares calculated would depend on the size of your portfolio.*

In [15]:
portfolio_value = input("Enter the size of your portfolio: ")
try:
    value = float(portfolio_value)
except ValueError:
    print("Please enter a valid number for your portfolio value.")
    portfolio_value = input("Enter the size of your portfolio: ")
    value = float(portfolio_value)
# We calculate the position_size by dividing the value of the portfolio by the number of stocks in the index.
position_size = value / len(stock_dataframe)
# We can then calculate the number of shares to buy per stock by dividing the position_size we want to take by the price of the stock.
for a in range(0, len(stock_dataframe)):
    current_stock = stock_dataframe.iloc[a]
    current_stock["Number of Shares to Buy"] = math.floor(position_size / current_stock['Stock Price'])
print(stock_dataframe)

'''LOGIC CHECK: The shares that have a lower price will have a higher number of shares to buy.
This is because you need a larger volume of cheaper shares if you want to match your for your portfolio size.'''

    Ticker Stock Price Market Capitalization Number of Shares to Buy
0        A      119.42           35273780186                   33428
1      AAL       16.26           10616154000                  245511
2      AAP       65.67            3903677761                   60789
3     AAPL      186.68         2936234089360                   21384
4     ABBV      135.92          239802253306                   29370
..     ...         ...                   ...                     ...
500    YUM       135.6           37979794895                   29439
501    ZBH      143.99           30031891347                   27724
502   ZBRA      264.33           13594581244                   15102
503   ZION        26.6            3939478647                  150075
504    ZTS      168.37           77805821012                   23709

[501 rows x 4 columns]


'LOGIC CHECK: The shares that have a lower price will have a higher number of shares to buy.\nThis is because you need a larger volume of cheaper shares if you want to match your for your portfolio size.'


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [50]:
writer = pd.ExcelWriter('Recommended_Trades.xlsx', engine='xlsxwriter')
stock_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [51]:
# Variables used for color scheme of a our excel sheet. Stores HTML HexCode for colors
background_color = '#0a0a23'
font_color = '#ffffff'
string_format = writer.book.add_format({'font_color':font_color, 'bg_color':background_color, 'border':1})
dollar_format = writer.book.add_format({'num_format': '$0.00', 'font_color':font_color, 'bg_color':background_color, 'border':1}) 
integer_format = writer.book.add_format({'num_format': '0', 'font_color':font_color, 'bg_color':background_color, 'border':1})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [48]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)

# Making the formats for the cells in the header row.
writer.sheets['Recommended Trades'].write('A1', 'Stocks', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [52]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.close()